In [38]:
def find_first_difference(file1_path, file2_path):
    with open(file1_path, 'r') as file1, open(file2_path, 'r') as file2:
        for line_num, (line1, line2) in enumerate(zip(file1, file2), start=1):
            if line1 != line2:
                return line_num, line1.strip(), line2.strip()
    return None


In [153]:
file1_path = "./debug_outs/nol2_l1_dm_short_gcc.out"
file2_path = "./pb_out/out.txt"
difference = find_first_difference(file1_path, file2_path)

if difference:
    line_num, line1, line2 = difference
    print(f"First difference found at line {line_num}:")
    print(f"File 1: {line1}")
    print(f"File 2: {line2}")
else:
    print("Files are identical.")
    



First difference found at line 1:
File 1: Cache Settings
File 2: L1 (15,5,0). LRU. Prefetcher disabled. L2 (17,5,2). LRU. +1 prefetcher. MIP.


In [154]:
num1 = 0xac1fd6dafe80
hex_str1 = hex(num1)
print("0xac1fd6dafe80 的十六进制表示为:", hex_str1)

num2 = 0x560feb6d7f70
hex_str2 = hex(num2)
print("0x560feb6d7f70 的十六进制表示为:", hex_str2)

num3 = num1 & 0xFFFFFFFFFFC0
num4 = num2 & 0xFFFFFFFFFFC0

0xac1fd6dafe80 的十六进制表示为: 0xac1fd6dafe80
0x560feb6d7f70 的十六进制表示为: 0x560feb6d7f70


In [2]:
config_list = [12.12,123,123,123]
total_cycle = 90908
total_mapping_efficiency = 90908

In [3]:
print("Config List: {}, Total Cycle: {}, Total Mapping Efficiency: {}".format(config_list, total_cycle, total_mapping_efficiency))

Config List: [12.12, 123, 123, 123], Total Cycle: 90908, Total Mapping Efficiency: 90908


In [285]:
def parse_cache_settings(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [l.rstrip("\n") for l in lines if len(l) >= 5]



In [264]:
file_dict

{' L1 (15,5,0). LRU. Prefetcher disabled. L2 (17,5,2). LRU. Prefetcher disabled.': '3.456',
 ' L1 (15,5,0). LRU. Prefetcher disabled. L2 (17,5,2). LRU. +1 prefetcher. MIP.': '3.235',
 ' L1 (15,5,0). LRU. Prefetcher disabled. L2 (17,5,2). LRU. Strided prefetcher. MIP.': '3.358',
 ' L1 (15,5,0). LRU. Prefetcher disabled. L2 (17,5,2). LRU. +1 prefetcher. LIP.': '3.230',
 ' L1 (15,5,0). LRU. Prefetcher disabled. L2 (17,5,2). LRU. Strided prefetcher. LIP.': '3.321',
 ' L1 (15,5,0). LRU. Prefetcher disabled. L2 (17,5,3). LRU. Prefetcher disabled.': '3.435',
 ' L1 (15,5,0). LRU. Prefetcher disabled. L2 (17,5,3). LRU. +1 prefetcher. MIP.': '3.213',
 ' L1 (15,5,0). LRU. Prefetcher disabled. L2 (17,5,3). LRU. Strided prefetcher. MIP.': '3.330',
 ' L1 (15,5,0). LRU. Prefetcher disabled. L2 (17,5,3). LRU. +1 prefetcher. LIP.': '3.207',
 ' L1 (15,5,0). LRU. Prefetcher disabled. L2 (17,5,3). LRU. Strided prefetcher. LIP.': '3.294',
 ' L1 (15,5,0). LRU. Prefetcher disabled. L2 (17,5,4). LRU. Prefetch

In [292]:
def parse_string(data_str):
    # Initialize a dictionary to hold the parsed values
    parsed_values = {
        'c1': None,
        'b1': None,
        's1': None,
        'c2': None,
        'b2': None,
        's2': None,
        'prefetch': None,
        'rp': None,
        'IP': None,
    }
    
    # Split the string by ". " to separate the sections
    sections = data_str.split('. ')
    
    # Extract values for c1, b1, s1 from the first section
    l1_values = sections[0].split('(')[1].split(')')[0].split(',')
    parsed_values['c1'] = int(l1_values[0])
    parsed_values['b1'] = int(l1_values[1])
    parsed_values['s1'] = int(l1_values[2])
    
    # Extract the replacement policy (rp) from the second section
    parsed_values['rp'] = sections[1].split('.')[0]
    
    # Extract values for c2, b2, s2 from the third section
    l2_values = sections[3].split('(')[1].split(')')[0].split(',')
    parsed_values['c2'] = int(l2_values[0])
    parsed_values['b2'] = int(l2_values[1])
    parsed_values['s2'] = int(l2_values[2])
    
    # Extract prefetch value
    prefetch_value = sections[5]
    parsed_values['prefetch'] = sections[5]
    
    # Extract IP from the last section
    parsed_values['IP'] = sections[-1]
    
    return parsed_values

# Test the function with the provided string

def compute_meta_data(c1,b1,s1,c2,b2,s2):
    meta1 = 2**(c1-b1)*(64-(c1-s1)+2)
    meta2 = 2**(c2-b2)*(64-(c2-s2)+1)
    return meta1 + meta2


from prettytable import PrettyTable

# Usage:
file_dict = {}
file_path = './pb_out/traces/mcf.trace.txt'
file_list = parse_cache_settings(file_path)
for i in range(0, len(file_list), 2):
    num_l = file_list[i+1].split()[0]
    file_dict[file_list[i]] = num_l

data_list = []
for line in file_dict.keys():
    line_dict = parse_string(line)
    parsed_data = [l for l in line_dict.values()]
    parsed_data.append(compute_meta_data(line_dict['c1'],line_dict['b1'],line_dict['s1'],line_dict['c2'],line_dict['b2'],line_dict['s2']))
    parsed_data.append(file_dict[line])
    data_list.append(parsed_data)
    
# Create a PrettyTable object
table = PrettyTable()

sorted_data_lists_by_last_element = sorted(data_list, key=lambda x: x[-1])

# Set column names based on the number of elements in the first list
column_names = [f"Column {i+1}" for i in range(len(data_list[0]))]
column_names = ["C1","B1", "S1","C2","B2","S2","Prefetch Policy","Replace Policy","Insertion Policy","Meta_cost","L1_AAT"]
table.field_names = column_names

# Add each list as a row in the table
for one_line in sorted_data_lists_by_last_element[:30]:
    table.add_row(one_line)

# Print the table
print(table)

+----+----+----+----+----+----+--------------------+----------------+------------------+-----------+--------+
| C1 | B1 | S1 | C2 | B2 | S2 |  Prefetch Policy   | Replace Policy | Insertion Policy | Meta_cost | L1_AAT |
+----+----+----+----+----+----+--------------------+----------------+------------------+-----------+--------+
| 15 | 7  | 2  | 17 | 7  | 5  |   +1 prefetcher    |      LRU       |       MIP.       |   67840   | 2.070  |
| 15 | 7  | 2  | 17 | 7  | 4  |   +1 prefetcher    |      LRU       |       MIP.       |   66816   | 2.071  |
| 15 | 7  | 2  | 17 | 7  | 3  |   +1 prefetcher    |      LRU       |       MIP.       |   65792   | 2.072  |
| 15 | 7  | 2  | 17 | 7  | 2  |   +1 prefetcher    |      LRU       |       MIP.       |   64768   | 2.075  |
| 15 | 7  | 2  | 17 | 7  | 3  |   +1 prefetcher    |      LRU       |       LIP.       |   65792   | 2.077  |
| 15 | 7  | 2  | 17 | 7  | 4  |   +1 prefetcher    |      LRU       |       LIP.       |   66816   | 2.080  |
| 15 | 7  

In [277]:
data_list

[[15,
  5,
  0,
  17,
  5,
  2,
  'Prefetcher disabled.',
  'LRU',
  'Prefetcher disabled.',
  257024,
  '3.456'],
 [15, 5, 0, 17, 5, 2, '+1 prefetcher', 'LRU', 'MIP.', 257024, '3.235'],
 [15, 5, 0, 17, 5, 2, 'Strided prefetcher', 'LRU', 'MIP.', 257024, '3.358'],
 [15, 5, 0, 17, 5, 2, '+1 prefetcher', 'LRU', 'LIP.', 257024, '3.230'],
 [15, 5, 0, 17, 5, 2, 'Strided prefetcher', 'LRU', 'LIP.', 257024, '3.321'],
 [15,
  5,
  0,
  17,
  5,
  3,
  'Prefetcher disabled.',
  'LRU',
  'Prefetcher disabled.',
  261120,
  '3.435'],
 [15, 5, 0, 17, 5, 3, '+1 prefetcher', 'LRU', 'MIP.', 261120, '3.213'],
 [15, 5, 0, 17, 5, 3, 'Strided prefetcher', 'LRU', 'MIP.', 261120, '3.330'],
 [15, 5, 0, 17, 5, 3, '+1 prefetcher', 'LRU', 'LIP.', 261120, '3.207'],
 [15, 5, 0, 17, 5, 3, 'Strided prefetcher', 'LRU', 'LIP.', 261120, '3.294'],
 [15,
  5,
  0,
  17,
  5,
  4,
  'Prefetcher disabled.',
  'LRU',
  'Prefetcher disabled.',
  265216,
  '3.425'],
 [15, 5, 0, 17, 5, 4, '+1 prefetcher', 'LRU', 'MIP.', 26521